In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

import sys
sys.path.append("/home/z5297792/UNSW-MRes/MRes/modules") 
from utils import dopioe, rossby_number, calc_tang_vel, find_directional_radii

df_eddies = pd.read_pickle(f'/srv/scratch/z5297792/Chapter2/SEACOFS_26yr_Eddy_Dataset/df_eddies_processed_1462_10650.pkl')
df_eddies


,Eddy,Day,Cyc,Lon,Lat,ic,jc,xc,yc,w,Q11,Q12,Q22,Rc,psi0,Age,fname
0,1,1462,CE,156.969773,-28.461100,179,274,505.897600,1353.012251,-0.000033,-0.008769,0.001546,-0.007521,69.500036,70.832145,57,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
1,1,1463,CE,156.993096,-28.433992,179,275,507.149717,1356.633687,-0.000030,-0.007934,0.000890,-0.007072,52.250752,156.388574,57,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
2,1,1464,CE,157.097940,-28.381485,181,276,515.130000,1365.650728,-0.000028,-0.006920,0.001050,-0.006852,67.000485,56.418837,57,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
3,1,1465,CE,157.094334,-28.315658,181,278,512.570772,1372.426342,-0.000033,-0.006959,0.002144,-0.009312,52.001786,31.355771,57,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
4,1,1466,CE,157.106324,-28.346479,181,277,514.728351,1369.599726,-0.000029,-0.005883,0.001179,-0.008429,56.251116,29.524244,57,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117257,2975,10646,AE,157.825620,-37.030656,248,97,850.502045,476.447416,0.000013,0.002921,0.000642,0.003759,72.751791,-76.302293,24,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
117258,2975,10647,AE,157.880796,-36.972725,249,98,853.608024,484.316677,0.000014,0.002819,0.000983,0.003948,74.250924,-47.064730,24,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
117259,2975,10648,AE,157.868516,-36.874363,248,100,849.759258,494.400614,0.000014,0.002968,0.000566,0.003910,75.750780,-79.414689,24,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
117260,2975,10649,AE,157.753058,-36.882303,246,99,840.037603,489.981707,0.000013,0.002804,0.000722,0.003863,76.000006,-92.084211,24,/srv/scratch/z3533156/26year_BRAN2020/outer_av...


In [2]:
import netCDF4 as nc
from scipy.interpolate import griddata
import netCDF4 as nc
from scipy.interpolate import RegularGridInterpolator
import time

# Field Data

fname = f'/srv/scratch/z3533156/26year_BRAN2020/outer_avg_01461.nc'

dataset = nc.Dataset(fname)

lon_rho = np.transpose(dataset.variables['lon_rho'], axes=(1, 0))
lat_rho = np.transpose(dataset.variables['lat_rho'], axes=(1, 0))
mask_rho = np.transpose(dataset.variables['mask_rho'], axes=(1, 0))
h =  np.transpose(dataset.variables['h'], axes=(1, 0))
angle = dataset.variables['angle'][0, 0]
z_r = np.load('/srv/scratch/z5297792/z_r.npy')
z_r = np.transpose(z_r, (1, 2, 0))[150, 150, :]

def distance(lat1, lon1, lat2, lon2):
    EARTH_RADIUS = 6357
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return EARTH_RADIUS * 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

j_mid = lon_rho.shape[1] // 2
i_mid = lon_rho.shape[0] // 2

dx = distance(lat_rho[:-1, j_mid], lon_rho[:-1, j_mid],
              lat_rho[1:, j_mid], lon_rho[1:, j_mid])
dy = distance(lat_rho[i_mid, :-1], lon_rho[i_mid, :-1],
              lat_rho[i_mid, 1:], lon_rho[i_mid, 1:])

x_grid = np.insert(np.cumsum(dx), 0, 0)
y_grid = np.insert(np.cumsum(dy), 0, 0)
X_grid, Y_grid = np.meshgrid(x_grid, y_grid, indexing='ij')


In [3]:
def compute_vert_data(df_eddies, eddy, X, Y, z_r, r=30):

    df = df_eddies[df_eddies.Eddy == eddy].copy()
    dic_data = {f'Eddy{eddy}': {}}
    fnumber_old = None
    x, y = X[:,0], Y[0,:]
    timer = 0

    # DOPIOE wont work if too close to boundary
    dx = np.max(np.diff(x))  # spacing in x-direction
    dy = np.max(np.diff(y))  # spacing in y-direction
    cell_size = np.max([dx, dy])        # max cell size in Euclidean units
    margin = int(np.ceil(r / cell_size))    

    for t, data in df.iterrows():
        day, fname, xc_surf, yc_surf, w_surf = data['Day'], data['fname'], data['xc'], data['yc'], data['w']
        fnumber = fname[-8:-3]

        # Only load if file changes
        if fnumber != fnumber_old:
            u_raw = np.load(f'/srv/scratch/z5297792/Climatology/u_v/u_{fnumber}.npy')
            v_raw = np.load(f'/srv/scratch/z5297792/Climatology/u_v/v_{fnumber}.npy')
            u_raw = np.where(np.abs(u_raw) > 1e30, np.nan, u_raw).astype(float)
            v_raw = np.where(np.abs(v_raw) > 1e30, np.nan, v_raw).astype(float)
            with nc.Dataset(fname) as dataset:
                ocean_time = dataset.variables['ocean_time'][:].data / 86400
            fnumber_old = fnumber
        t_rel = np.where(ocean_time == day)[0][0]

        u_t, v_t = u_raw[:, :, :, t_rel], v_raw[:, :, :, t_rel]

        df_data = []
        xc, yc = xc_surf, yc_surf
        xc_old, yc_old = xc_surf, yc_surf

        # for k in range(24):
        for k in range(u_t.shape[-1]):
            u, v = u_t[:, :, k], v_t[:, :, k]

            R_grid = np.hypot(xc - X, yc - Y)
            ic, jc = map(int, np.unravel_index(np.argmin(R_grid), R_grid.shape))
            
            if (ic < margin or ic >= X.shape[0] - margin or
                jc < margin or jc >= X.shape[1] - margin):
                break

            # horizontal transect (constant y = y[jc])
            x_mask = np.abs(x - xc) <= r
            x1 = x[x_mask]
            y1 = np.full_like(x1, y[jc])
            u1 = u[x_mask, jc]
            v1 = v[x_mask, jc]
        
            # vertical transect (constant x = x[ic])
            y_mask = np.abs(y - yc) <= r
            y2 = y[y_mask]
            x2 = np.full_like(y2, x[ic])
            u2 = u[ic, y_mask]
            v2 = v[ic, y_mask]

            xc, yc, w, Q, _, psi0, _ = dopioe(x1, y1, u1, v1, x2, y2, u2, v2)

            if (np.sign(w) != np.sign(w_surf)) or (np.hypot(xc_old - xc, yc_old - yc) > 50):
                break
            
            xc_old, yc_old = xc, yc

            w *= 1e-3 # to s^-1
            # Ro = rossby_number(w, yc)

            radii = find_directional_radii(u, v, X, Y, xc, yc, calc_tang_vel)
            Rc = np.mean([radii['up'], radii['right'], radii['down'], radii['left']])

            df_data.append({
                'x': xc, 'y': yc, 'Q': Q, 'w': w,
                'Rc': Rc, 'psi0': psi0,
                'z': k, 'Depth': z_r[k]
            })

        if df_data:
            df_data = pd.DataFrame(df_data)
            dx = df_data['x'].diff()
            dy = df_data['y'].diff()
            df_data['TD'] = np.hypot(df_data['x'] - df_data.iloc[0]['x'],
                                  df_data['y'] - df_data.iloc[0]['y'])
            dic_data[f'Eddy{eddy}'][f'Day{day}'] = df_data
        else:
            dic_data[f'Eddy{eddy}'][f'Day{day}'] = pd.DataFrame(columns=['x', 'y', 'Q', 'w', 'Rc', 'psi0', 'z','Depth'])

        # if timer % 10 == 0:
        #     print(f'{timer / len(df) * 100:.1f}%')
        #     timer += 1
        
    return dic_data
    

In [4]:
dic_vert_eddies = {}
for e, eddy in enumerate(df_eddies.Eddy.unique()):
    if eddy >= 2608:
        dic = compute_vert_data(df_eddies, eddy, X_grid, Y_grid, z_r)
        dic_vert_eddies = dic_vert_eddies | dic 
    
        with open(f"/srv/scratch/z5297792/Chapter2/SEACOFS_26yr_Eddy_Dataset/Vertical_Eddy_Dataset/dic_vert_eddies_{2608}_{df_eddies.Eddy.max()}.pkl", "wb") as f:
            pickle.dump(dic_vert_eddies, f)
    
        if e % 10 == 0:
            print(f'{e+1}/{len(df_eddies.Eddy.unique())}')


2611/2975
2621/2975


/home/z5297792/UNSW-MRes/MRes/modules/utils.py:388: RuntimeWarning: overflow encountered in square
  s = np.sum((ui - u_pred)**2 + (vi - v_pred)**2)


2631/2975


/home/z5297792/UNSW-MRes/MRes/modules/utils.py:388: RuntimeWarning: overflow encountered in square
  s = np.sum((ui - u_pred)**2 + (vi - v_pred)**2)


2641/2975
2651/2975
2661/2975


...

2921/2975
2931/2975
2941/2975


/home/z5297792/UNSW-MRes/MRes/modules/utils.py:386: RuntimeWarning: overflow encountered in multiply
  u_pred = -beta * E
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:387: RuntimeWarning: overflow encountered in multiply
  v_pred =  alpha * E
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:388: RuntimeWarning: overflow encountered in square
  s = np.sum((ui - u_pred)**2 + (vi - v_pred)**2)
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:388: RuntimeWarning: overflow encountered in square
  s = np.sum((ui - u_pred)**2 + (vi - v_pred)**2)


2951/2975


/home/z5297792/UNSW-MRes/MRes/modules/utils.py:387: RuntimeWarning: overflow encountered in multiply
  v_pred =  alpha * E
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:388: RuntimeWarning: overflow encountered in square
  s = np.sum((ui - u_pred)**2 + (vi - v_pred)**2)


2961/2975
2971/2975
